<a href="https://colab.research.google.com/github/AndrewSLowe/AndrewSLowe.github.io/blob/master/module3/2_1_3A_regression_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 3

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `RidgeRegression`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `RidgeRegression`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [131]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')

Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification
 * branch            master     -> FETCH_HEAD
Already up to date.


In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv('../data/condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [213]:
df = df[(df['SALE_PRICE'] > 100000) & (df['SALE_PRICE'] < 2000000) & (df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS')]

print(df.shape)
df.head()

(3151, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


In [0]:
import pandas_profiling
pandas_profiling.ProfileReport(df)

In [215]:
df.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                   object
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

In [0]:
df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].str.replace(',', '')
df = df.dropna(subset=['LAND_SQUARE_FEET'])

In [0]:
df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].astype(int)


In [0]:
df = df[df['LAND_SQUARE_FEET'] > 0]


In [219]:
df.isnull().sum()

BOROUGH                              0
NEIGHBORHOOD                         0
BUILDING_CLASS_CATEGORY              0
TAX_CLASS_AT_PRESENT                 0
BLOCK                                0
LOT                                  0
EASE-MENT                         3111
BUILDING_CLASS_AT_PRESENT            0
ADDRESS                              0
APARTMENT_NUMBER                  3110
ZIP_CODE                             0
RESIDENTIAL_UNITS                    0
COMMERCIAL_UNITS                     0
TOTAL_UNITS                          0
LAND_SQUARE_FEET                     0
GROSS_SQUARE_FEET                    0
YEAR_BUILT                           0
TAX_CLASS_AT_TIME_OF_SALE            0
BUILDING_CLASS_AT_TIME_OF_SALE       0
SALE_PRICE                           0
SALE_DATE                            0
dtype: int64

In [220]:
df.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                    int64
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

In [0]:
from sklearn.preprocessing import scale

num_cols = df.select_dtypes(include='number').columns
df[num_cols] = scale(df[num_cols])

In [222]:
df.shape

(3111, 21)

Dropping values with lots of nans and high cardinality.

In [0]:
#High NaN values
df = df.drop(columns=['EASE-MENT', 'APARTMENT_NUMBER'])

In [224]:
print(df.shape)
df.head()

(3111, 19)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,-0.346433,4.540184,A9,4832 BAY PARKWAY,0.429242,-0.017932,-0.125285,-0.1266,2.065749,-0.290310,-0.516335,0.0,A9,-0.279997,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,0.276715,-0.013511,A1,80-23 232ND STREET,0.839245,-0.017932,-0.125285,-0.1266,0.480139,0.904793,-0.143180,0.0,A1,-1.475738,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,-0.676910,-0.344575,A1,1260 RHINELANDER AVE,-1.171226,-0.017932,-0.125285,-0.1266,0.196994,0.979045,-0.702912,0.0,A1,0.608269,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,-0.419215,-0.032250,A1,469 E 25TH ST,0.420917,-0.017932,-0.125285,-0.1266,0.480139,2.105199,-1.673113,0.0,A1,-1.731969,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,0.279802,0.292567,A5,5521 WHITTY LANE,0.373048,-0.017932,-0.125285,-0.1266,-0.816664,0.676733,-0.143180,0.0,A5,-0.040848,01/02/2019


In [225]:
df.select_dtypes(include='number').describe().T

,count,mean,std,min,25%,50%,75%,max
BLOCK,3111.0,-5.331640e-17,1.000161,-1.754239,-0.740691,-0.147890,0.830682,2.435489
LOT,3111.0,6.923281e-18,1.000161,-0.457012,-0.332082,-0.207152,-0.032250,16.527208
ZIP_CODE,3111.0,-1.074179e-17,1.000161,-2.068238,-1.171226,0.439648,0.810107,1.394934
RESIDENTIAL_UNITS,3111.0,4.881025e-16,1.000161,-0.017932,-0.017932,-0.017932,-0.017932,55.767374
COMMERCIAL_UNITS,3111.0,4.713719e-16,1.000161,-0.125285,-0.125285,-0.125285,-0.125285,15.465223
TOTAL_UNITS,3111.0,4.824884e-17,1.000161,-0.126600,-0.126600,-0.126600,-0.126600,15.318646
LAND_SQUARE_FEET,3111.0,-4.582213e-17,1.000161,-1.589649,-0.652440,-0.326257,0.480139,8.921249
GROSS_SQUARE_FEET,3111.0,2.640839e-18,1.000161,-2.137769,-0.596157,-0.221362,0.354975,11.289459
YEAR_BUILT,3111.0,2.056910e-17,1.000161,-2.008952,-0.702912,-0.143180,0.416551,2.767424
TAX_CLASS_AT_TIME_OF_SALE,3111.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [226]:
df.select_dtypes(exclude='number').describe().T

,count,unique,top,freq
BOROUGH,3111,5,4,1553
NEIGHBORHOOD,3111,6,OTHER,2950
BUILDING_CLASS_CATEGORY,3111,1,01 ONE FAMILY DWELLINGS,3111
TAX_CLASS_AT_PRESENT,3111,1,1,3111
BUILDING_CLASS_AT_PRESENT,3111,12,A1,1185
ADDRESS,3111,3095,46-12 30TH ROAD,2
BUILDING_CLASS_AT_TIME_OF_SALE,3111,10,A1,1186
SALE_DATE,3111,91,01/31/2019,77


In [0]:
target = 'SALE_PRICE'
high_cardinality = ['ADDRESS', 'TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_CATEGORY', 'SALE_DATE']
features = train.columns.drop([target] + high_cardinality)

In [0]:
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
cutoff = pd.to_datetime('2019-4-01')
train = df[df.SALE_DATE < cutoff]
test = df[df.SALE_DATE >= cutoff]

In [0]:
# ADDRESS has a high cardinality: 22691 distinct values	Warning
# BUILDING_CLASS_AT_PRESENT has a high cardinality: 147 distinct values	Warning
# BUILDING_CLASS_AT_TIME_OF_SALE has a high cardinality: 147 distinct values

target = 'SALE_PRICE'
high_cardinality = ['ADDRESS', 'BUILDING_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_CATEGORY', 'SALE_DATE']
features = train.columns.drop([target] + high_cardinality)

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [0]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)           
X_test = encoder.transform(X_test)

In [235]:
print(train.shape)
print(test.shape)


(2476, 19)
(635, 19)


In [236]:
X_train.head()

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,TAX_CLASS_AT_PRESENT_1,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
44,1,0,0,0,0,1,0,0,0,0,0,1,-0.346433,4.540184,0.429242,-0.017932,-0.125285,-0.1266,2.065749,-0.290310,-0.516335,0.0
61,0,1,0,0,0,1,0,0,0,0,0,1,0.276715,-0.013511,0.839245,-0.017932,-0.125285,-0.1266,0.480139,0.904793,-0.143180,0.0
78,0,0,1,0,0,1,0,0,0,0,0,1,-0.676910,-0.344575,-1.171226,-0.017932,-0.125285,-0.1266,0.196994,0.979045,-0.702912,0.0
108,1,0,0,0,0,1,0,0,0,0,0,1,-0.419215,-0.032250,0.420917,-0.017932,-0.125285,-0.1266,0.480139,2.105199,-1.673113,0.0
111,1,0,0,0,0,1,0,0,0,0,0,1,0.279802,0.292567,0.373048,-0.017932,-0.125285,-0.1266,-0.816664,0.676733,-0.143180,0.0


In [251]:
from sklearn.feature_selection import f_regression, SelectKBest

selector = SelectKBest(score_func=f_regression, k=19)           #Chose 15 arbitrarily.


X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
X_train_selected.shape, X_test_selected.shape

((2476, 19), (635, 19))

In [252]:
all_names = X_train.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)
    
print('\n')
print('Features not selected:')
for name in unselected_names:
    print(name)

Features selected:
BOROUGH_3
BOROUGH_4
BOROUGH_2
BOROUGH_5
BOROUGH_1
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_BEDFORD STUYVESANT
NEIGHBORHOOD_FOREST HILLS
NEIGHBORHOOD_BOROUGH PARK
NEIGHBORHOOD_ASTORIA
BLOCK
LOT
ZIP_CODE
COMMERCIAL_UNITS
TOTAL_UNITS
LAND_SQUARE_FEET
GROSS_SQUARE_FEET
YEAR_BUILT


Features not selected:
TAX_CLASS_AT_PRESENT_1
RESIDENTIAL_UNITS
TAX_CLASS_AT_TIME_OF_SALE


In [246]:
X_train.corr()

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,TAX_CLASS_AT_PRESENT_1,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
BOROUGH_3,1.000000e+00,-4.186122e-01,-1.407185e-01,-2.643803e-01,-8.796931e-03,4.665328e-04,-7.840595e-02,1.300929e-01,-3.638848e-02,1.594599e-01,-2.923524e-02,NaN,-3.291700e-02,6.081132e-02,2.071952e-01,3.163154e-16,7.073013e-02,7.073013e-02,-2.240776e-01,2.361971e-02,-2.218889e-01,NaN
BOROUGH_4,-4.186122e-01,1.000000e+00,-3.075570e-01,-5.778345e-01,-1.922673e-02,-1.672061e-01,1.872997e-01,-5.445848e-02,8.692648e-02,-6.675186e-02,6.983848e-02,NaN,5.698349e-01,-9.406273e-02,7.901327e-01,-1.887975e-16,2.540245e-03,2.540245e-03,4.486533e-02,-7.123740e-02,-2.499155e-01,NaN
BOROUGH_2,-1.407185e-01,-3.075570e-01,1.000000e+00,-1.942419e-01,-6.463161e-03,7.414148e-02,-5.760535e-02,-1.830649e-02,-2.673485e-02,-2.243897e-02,-2.147932e-02,NaN,-1.737889e-01,2.194353e-02,-3.425707e-01,-3.762175e-16,1.678328e-03,1.678328e-03,-8.729014e-02,4.612892e-02,-7.824035e-02,NaN
BOROUGH_5,-2.643803e-01,-5.778345e-01,-1.942419e-01,1.000000e+00,-1.214291e-02,1.392961e-01,-1.082282e-01,-3.439401e-02,-5.022912e-02,-4.215806e-02,-4.035508e-02,NaN,-5.003317e-01,3.952478e-02,-8.371648e-01,-2.304730e-16,-6.255413e-02,-6.255413e-02,1.939526e-01,3.050732e-02,5.183832e-01,NaN
BOROUGH_1,-8.796931e-03,-1.922673e-02,-6.463161e-03,-1.214291e-02,1.000000e+00,4.634908e-03,-3.601162e-03,-1.144418e-03,-1.671312e-03,-1.402758e-03,-1.342766e-03,NaN,-2.302542e-02,3.790680e-02,-2.151829e-02,3.389809e-16,-2.545907e-03,-2.545907e-03,-2.545699e-02,-1.810342e-03,-1.472936e-02,NaN
NEIGHBORHOOD_OTHER,4.665328e-04,-1.672061e-01,7.414148e-02,1.392961e-01,4.634908e-03,1.000000e+00,-7.769652e-01,-2.469129e-01,-3.605923e-01,-3.026506e-01,-2.897071e-01,NaN,1.271620e-01,4.955240e-02,-1.537397e-01,-4.263362e-16,-4.134304e-02,-4.134304e-02,1.852083e-02,-3.450446e-02,5.330606e-02,NaN
NEIGHBORHOOD_FLUSHING-NORTH,-7.840595e-02,1.872997e-01,-5.760535e-02,-1.082282e-01,-3.601162e-03,-7.769652e-01,1.000000e+00,-1.020006e-02,-1.489620e-02,-1.250260e-02,-1.196790e-02,NaN,-5.422421e-02,-4.234671e-02,1.343931e-01,5.346720e-16,1.290180e-02,1.290180e-02,2.995554e-02,-5.982292e-03,-6.201591e-03,NaN
NEIGHBORHOOD_BEDFORD STUYVESANT,1.300929e-01,-5.445848e-02,-1.830649e-02,-3.439401e-02,-1.144418e-03,-2.469129e-01,-1.020006e-02,1.000000e+00,-4.733884e-03,-3.973221e-03,-3.803297e-03,NaN,-7.193940e-02,-6.938693e-03,2.646996e-02,-5.550706e-16,-7.211117e-03,-7.211117e-03,-4.000996e-02,3.165030e-02,-4.879033e-02,NaN
NEIGHBORHOOD_FOREST HILLS,-3.638848e-02,8.692648e-02,-2.673485e-02,-5.022912e-02,-1.671312e-03,-3.605923e-01,-1.489620e-02,-4.733884e-03,1.000000e+00,-5.802504e-03,-5.554348e-03,NaN,-7.758902e-02,-1.875726e-02,6.515377e-02,4.470487e-16,-1.053114e-02,-1.053114e-02,-1.194649e-02,4.813476e-02,-3.393295e-02,NaN
NEIGHBORHOOD_BOROUGH PARK,1.594599e-01,-6.675186e-02,-2.243897e-02,-4.215806e-02,-1.402758e-03,-3.026506e-01,-1.250260e-02,-3.973221e-03,-5.802504e-03,1.000000e+00,-4.661848e-03,NaN,-1.798727e-02,-1.703545e-02,3.148758e-02,3.753294e-17,3.564322e-02,3.564322e-02,-4.777513e-02,1.401469e-02,-4.578812e-02,NaN


28724626712.507473


In [258]:
alphas = []      #alpha can be more than 1. We're looking at alpha from 0 to 200.
mses = []

for alpha in range(0, 200, 1):
    ridge_reg_split = Ridge(alpha=alpha).fit(X_train, y_train)
    mse = mean_squared_error(y_test, ridge_reg_split.predict(X_test))
    print(alpha, mse)
    alphas.append(alpha)
    mses.append(mse)

0 1.460675855774525e+25
1 0.5929617060096428
2 0.5938629580849684
3 0.5947337873974301
4 0.5955692468267576
5 0.5963639601276634
6 0.597115721818898
7 0.59782472917742
8 0.5984925854533453
9 0.5991216077823144
10 0.5997144062531886
11 0.6002736425428452
12 0.6008018982226948
13 0.6013016079104646
14 0.601775030017549
15 0.6022242388470406
16 0.6026511284377539
17 0.6030574225087493
18 0.6034446872134738
19 0.6038143448135785
20 0.6041676872161851
21 0.604505888813458
22 0.6048300183554255
23 0.6051410497569724
24 0.6054398718364047
25 0.6057272970352717
26 0.6060040691950308
27 0.6062708704765004
28 0.6065283275094576
29 0.6067770168562129
30 0.6070174698669607
31 0.6072501769976067
32 0.6074755916534343
33 0.6076941336148931
34 0.6079061920951885
35 0.6081121284733524
36 0.6083122787410957
37 0.6085069556969873
38 0.6086964509173074
39 0.6088810365292548
40 0.6090609668089877
41 0.6092364796241787
42 0.6094077977383464
43 0.6095751299921057
44 0.6097386723746454
45 0.6098986089971459
